In [1]:
import dace
import cupy as cp

from IPython.display import Code
from dace.config import Config


print(Config.get('compiler', 'cuda', 'implementation'))


experimental


In [2]:

# Generate framework
sdfg = dace.SDFG("Warp_test_1")

state = sdfg.add_state("main")

# Generate access nodes
a_dev = sdfg.add_array("A", (32,), dace.uint32, dace.dtypes.StorageType.GPU_Global)
b_dev = sdfg.add_array("B", (32,), dace.uint32, dace.dtypes.StorageType.GPU_Global)
a_acc = state.add_access("A")
b_acc = state.add_access("B")


# Generate maps, connect entries with access data
gpu_map_entry, gpu_map_exit = state.add_map(name = "GPU_Map",
                                            ndrange = dict(i='0:32:32'),
                                            schedule = dace.dtypes.ScheduleType.GPU_Device)
state.add_edge(a_acc, None, gpu_map_entry, None, dace.memlet.Memlet('A[0:32]'))


tblock_map_entry, tblock_map_exit = state.add_map(name = "Block_Map",
                                                  ndrange = dict(j='0:32'),
                                                  schedule = dace.dtypes.ScheduleType.GPU_ThreadBlock)
state.add_edge(gpu_map_entry, None, tblock_map_entry, None, dace.memlet.Memlet('A[0:32]'))




tasklet, warp_scope_entry, warp_scope_exit = state.add_mapped_tasklet(
    name='WarpLevel_Operation',
    map_ranges=dict(_='0:1'),
    inputs=dict(inp=dace.Memlet('A[0:32]', volume=32)),
    code=
''' 
value = inp[j]
out = __reduce_add_sync(0xFFFFFFFF, value);
''',
    outputs=dict(out=dace.Memlet("B[j]")),
    schedule=dace.dtypes.ScheduleType.GPU_Warp
)

state.add_edge(tblock_map_entry, None, warp_scope_entry, None, dace.memlet.Memlet('A[0:32]'))

# Connect Exit nodes
state.add_edge(warp_scope_exit, None, tblock_map_exit, None, dace.memlet.Memlet('B[j]'))
state.add_edge(tblock_map_exit, None, gpu_map_exit, None, dace.memlet.Memlet('B[j]'))
state.add_edge(gpu_map_exit, None, b_acc, None, dace.memlet.Memlet('B[0:32]'))




#sdfg.fill_scope_connectors()



sdfg

SDFG (Warp_test_1)

In [3]:
Code(sdfg.generate_code()[1].clean_code, language='cpp')

#include <cuda_runtime.h>
#include <dace/dace.h>


struct Warp_test_1_state_t {
    dace::cuda::Context *gpu_context;
};



DACE_EXPORTED int __dace_init_experimental_cuda(Warp_test_1_state_t *__state);
DACE_EXPORTED int __dace_exit_experimental_cuda(Warp_test_1_state_t *__state);



int __dace_init_experimental_cuda(Warp_test_1_state_t *__state) {
    int count;

    // Check that we are able to run cuda code
    if (cudaGetDeviceCount(&count) != cudaSuccess)
    {
        printf("ERROR: GPU drivers are not configured or cuda-capable device "
               "not found\n");
        return 1;
    }
    if (count == 0)
    {
        printf("ERROR: No cuda-capable devices found\n");
        return 2;
    }

    // Initialize cuda before we run the application
    float *dev_X;
    DACE_GPU_CHECK(cudaMalloc((void **) &dev_X, 1));
    DACE_GPU_CHECK(cudaFree(dev_X));

    

    __state->gpu_context = new dace::cuda::Context(1, 1);

    // Create cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamCreateWithFlags(&__state->gpu_context->internal_streams[i], cudaStreamNonBlocking));
        __state->gpu_context->streams[i] = __state->gpu_context->internal_streams[i]; // Allow for externals to modify streams
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventCreateWithFlags(&__state->gpu_context->events[i], cudaEventDisableTiming));
    }

    

    return 0;
}

int __dace_exit_experimental_cuda(Warp_test_1_state_t *__state) {
    

    // Synchronize and check for CUDA errors
    int __err = static_cast<int>(__state->gpu_context->lasterror);
    if (__err == 0)
        __err = static_cast<int>(cudaDeviceSynchronize());

    // Destroy cuda streams and events
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaStreamDestroy(__state->gpu_context->internal_streams[i]));
    }
    for(int i = 0; i < 1; ++i) {
        DACE_GPU_CHECK(cudaEventDestroy(__state->gpu_context->events[i]));
    }

    delete __state->gpu_context;
    return __err;
}

DACE_EXPORTED bool __dace_gpu_set_stream(Warp_test_1_state_t *__state, int streamid, gpuStream_t stream)
{
    if (streamid < 0 || streamid >= 1)
        return false;

    __state->gpu_context->streams[streamid] = stream;

    return true;
}

DACE_EXPORTED void __dace_gpu_set_all_streams(Warp_test_1_state_t *__state, gpuStream_t stream)
{
    for (int i = 0; i < 1; ++i)
        __state->gpu_context->streams[i] = stream;
}

__global__ void GPU_Map_0_0_2(dace::uint * __restrict__ A, dace::uint * __restrict__ B)
{ // Kernel scope (open 1)
    int i = (32 * blockIdx.x);
    { // ThreadBlock Scope (open 1)
        int j = threadIdx.x;
        { // WarpLevel Scope (open 1)

            int warpId = (threadIdx.x + blockDim.x * threadIdx.y + blockDim.x * blockDim.y * threadIdx.z) / 32;
            if ( 0 <= warpId && warpId <= 0) { // WarpLevel Scope (open 2)



                //////////////////////////////////////
                // WarpLevel operations start
                //////////////////////////////////////
                {
                    dace::uint* inp = &A[0];
                    dace::uint out;

                    ///////////////////
                    // Tasklet code (WarpLevel_Operation)
                    auto value = inp[j];
                    out = __reduce_add_sync(4294967295U, value);
                    ///////////////////

                    B[j] = out;
                }
                //////////////////////////////////////
                // WarpLevel operations end
                //////////////////////////////////////



            } // WarpLevel Scope (close 1)
        } // WarpLevel Scope (close 2)
    } // ThreadBlock Scope (close 1)
} // Kernel scope (close 1)


DACE_EXPORTED void __dace_runkernel_GPU_Map_0_0_2(Warp_test_1_state_t *__state, dace::uint * __restrict__ A, dace::uint * __restrict__ B);
void __dace_runkernel_GPU_Map_0_0_2(Warp_test_1_state_t *__state, dace::uint * __restrict__ A, dace::uint

In [4]:
call_it = sdfg.compile()

In [5]:
A = cp.ones(32, dtype=cp.uint32) 
B = cp.zeros(32, dtype=cp.uint32) 

sdfg(A=A, B=B)

print(B)

/home/berkay/master-thesis/dace/dace/sdfg/sdfg.py:2373: UserWarning: SDFG 'Warp_test_1' is already loaded by another object, recompiling under a different name 'Warp_test_1_0'.
  warnings.warn(f"SDFG '{self.name}' is already loaded by another object, recompiling under a different "


[32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
 32 32 32 32 32 32 32 32]
